In [4]:
# Importamos las librerías que necesitamos

# Librerías de extracción de datos
# -----------------------------------------------------------------------

# Importaciones:
# Beautifulsoup
from bs4 import BeautifulSoup

# Requests
import requests

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar 


# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

# Librerías para tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

import sys
sys.path.append("../")
from src import soporte_eco
pd.set_option('display.max_rows', None)
from collections import Counter
import random


In [5]:
#Vamos a scrapear ecoalf

#Accedemos a la página
driver = webdriver.Chrome()
url = "https://ecoalf.com/collections/abrigos-y-chaquetas-210-hombre"
driver.get(url)
driver.maximize_window()
sleep(10)
driver.execute_script(f"window.scrollTo(0, {0,20});")

WebDriverWait(driver,5).until(EC.presence_of_element_located(("xpath","/html/body/div[1]/div/div[1]/div/div[2]/button[2]"))).click()
print("Hemos clickado en la x")

#Scrolleamos hasta el final para que cargue toda la página, de 200 en 200 para que cargue tranquilamente
altura_pagina = driver.execute_script("return document.body.scrollHeight")
for y in range(0, altura_pagina, 200):
    driver.execute_script(f"window.scrollTo(0, {y});")
    altura_pagina = driver.execute_script("return document.body.scrollHeight")
    sleep(2)


# Usamos BeautifulSoup para analizar el HTML de la página cargada
sopa_chaquetas_ecoalf = BeautifulSoup(driver.page_source, "html.parser")

# Extraemos todos los enlaces de chaquetas
nombre_chaqueta = sopa_chaquetas_ecoalf.find_all("a", class_="full-unstyled-link")

# Mostramos la cantidad de chaquetas encontradas
print(f"Cantidad de chaquetas encontradas: {len(nombre_chaqueta)}")

# Cerramos el navegador
driver.quit()


# #Sacamos las categorías y género
denominacion_url = url.rsplit("-")
genero = denominacion_url[-1]
categoria = denominacion_url[-3]


nombre_chaqueta = sopa_chaquetas_ecoalf.findAll("a", class_= "full-unstyled-link")

precios_chaquetas = sopa_chaquetas_ecoalf.find_all("div", class_="price__regular price-item price-item--regular")

#Le quitamos los headers a nombre_chaqueta
nombre_chaqueta = nombre_chaqueta[14:]
print(f"Hay {len(nombre_chaqueta)} chaquetas")
print(f"Hay {len(precios_chaquetas)} precios")


Hemos clickado en la x
Cantidad de chaquetas encontradas: 57
Hay 43 chaquetas
Hay 36 precios


In [6]:

# Buscar todos los elementos <a> con la clase "full-unstyled-link"
enlaces = sopa_chaquetas_ecoalf.find_all("a", class_="full-unstyled-link")

# Extraer el atributo 'href' de cada enlace
links = [enlace['href'] for enlace in enlaces if 'href' in enlace.attrs]
links = links[14:]

links_completos = ["https://ecoalf.com" + i for i in links]

In [7]:
links_completos

['https://ecoalf.com/products/venuealf-jacket-man-deep-navy-aw24',
 'https://ecoalf.com/products/graysalf-jacket-man-deep-navy-aw24',
 'https://ecoalf.com/products/livornoalf-jacket-man-black-aw24',
 'https://ecoalf.com/products/velanalf-jacket-man-deep-navy-aw24',
 'https://ecoalf.com/products/pasualf-jacket-man-deep-navy-aw24',
 'https://ecoalf.com/products/venuealf-jacket-man-woodland-green-aw24',
 'https://ecoalf.com/products/vinsonalf-jacket-man-dark-olive-aw24',
 'https://ecoalf.com/products/new-icebergalf-jacket-man-woodland-green-aw24',
 'https://ecoalf.com/products/parkoalf-jacket-man-deepnavy-aw24',
 'https://ecoalf.com/products/vinsonalf-jacket-man-deep-navy-aw24',
 'https://ecoalf.com/products/cartesalf-jacket-man-dark-olive-aw24',
 'https://ecoalf.com/products/new-vintagealf-jacket-man-anthracite-aw24',
 'https://ecoalf.com/products/graysalf-jacket-man-armygreen-aw24',
 'https://ecoalf.com/products/new-icebergalf-jacket-man-deep-navy-aw24',
 'https://ecoalf.com/products/pa

In [8]:
nombre_elemento =  []
precio_elemento =  []
detalles_elemento =  []

for single_link in links_completos:
    res_link = requests.get(single_link)
    sopa_elemento = BeautifulSoup(res_link.content, "html.parser")
    nombre_elemento.append(sopa_elemento.find_all("h1","product__title h4"))
    precio_elemento.append(sopa_elemento.find_all("div", class_="price__regular price-item price-item--regular"))
    detalles_elemento.append(sopa_elemento.find_all("div", class_="product__description rte"))

In [9]:
nombres_done = []
for lista in nombre_elemento:
    for i in lista:
        nombres_done.append(i.getText().strip())

nombre_elemento

[[<h1 class="product__title h4">
                      CHAQUETA VENUE AZUL MARINO
                    </h1>,
  <h1 class="product__title h4">
                              CHAQUETA VENUE AZUL MARINO
                            </h1>],
 [<h1 class="product__title h4">
                      CHAQUETA GRAYS AZUL MARINO
                    </h1>,
  <h1 class="product__title h4">
                              CHAQUETA GRAYS AZUL MARINO
                            </h1>],
 [<h1 class="product__title h4">
                      ABRIGO LIVORNO NEGRO
                    </h1>,
  <h1 class="product__title h4">
                              ABRIGO LIVORNO NEGRO
                            </h1>],
 [<h1 class="product__title h4">
                      ABRIGO VELAN AZUL MARINO
                    </h1>,
  <h1 class="product__title h4">
                              ABRIGO VELAN AZUL MARINO
                            </h1>],
 [<h1 class="product__title h4">
                      ABRIGO PASU AZUL MARI

In [10]:
precios_done = []
for lista in precio_elemento:
    for i in lista:
        precios_done.append(i.getText().strip())

In [29]:
detalles_element = []
for lista in detalles_elemento:
    for i in lista:
        detalles_element.append(i.getText().strip())

nueva_lista = []
for i in detalles_element:
        
    nueva_lista.append(i.split("\n"))
  

descripcion = []
tejido = []
for i in nueva_lista:
    descripcion.append(i[0])
    tejido.append(i[2])



In [12]:
len(tejido)

43

In [13]:
data = {"nombre": nombres_done,
        "precios": precios_done
        }
df_ecoalf = pd.DataFrame(data)



In [14]:
df_reemplazo = df_ecoalf["nombre"]

In [15]:
def split_marca(text):
    palabra = text.split()
    return " ".join(palabra[:2])

In [16]:
df_reemplazo.apply(split_marca)

0       CHAQUETA VENUE
1       CHAQUETA VENUE
2       CHAQUETA GRAYS
3       CHAQUETA GRAYS
4       ABRIGO LIVORNO
5       ABRIGO LIVORNO
6         ABRIGO VELAN
7         ABRIGO VELAN
8          ABRIGO PASU
9          ABRIGO PASU
10      CHAQUETA VENUE
11      CHAQUETA VENUE
12     CHAQUETA VINSON
13     CHAQUETA VINSON
14      ABRIGO ICEBERG
15      ABRIGO ICEBERG
16        ABRIGO PARKO
17        ABRIGO PARKO
18     CHAQUETA VINSON
19     CHAQUETA VINSON
20     CHAQUETA CARTES
21     CHAQUETA CARTES
22          ABRIGO NEW
23          ABRIGO NEW
24      CHAQUETA GRAYS
25      CHAQUETA GRAYS
26      ABRIGO ICEBERG
27      ABRIGO ICEBERG
28        ABRIGO PARKO
29        ABRIGO PARKO
30        CHAQUETA ASP
31        CHAQUETA ASP
32        CHAQUETA ASP
33        CHAQUETA ASP
34     CHAQUETA CRONIN
35     CHAQUETA CRONIN
36        ABRIGO PARKO
37        ABRIGO PARKO
38       CHAQUETA LAWU
39       CHAQUETA LAWU
40      CHALECO AORAKI
41      CHALECO AORAKI
42      CHAQUETA ALPEN
43      CHA

In [17]:
descripcion

['La chaqueta Venue está hecha de poliéster reciclado que procede de botellas de plástico a las que damos una segunda vida. Lleva capucha incorporada ajustable y entre sus propiedades técnicas destaca que es impermeable y transpirable; la chaqueta perfecta para los días de lluvia.',
 'La chaqueta Grays es monomaterial y está hecha de 100% poliéster reciclado procedente de botellas de plástico. Además, es transpirable y repelente al agua. Una chaqueta muy cómoda y perfecta para este invierno con la que combinarás estilo y sostenibilidad.',
 'El abrigo Livorno está hecho con nuestro Hilo del Mar procedente de botellas de plástico recicladas del fondo del mar a través del proyecto Upcycling the Oceans de la Fundación Ecoalf. Hemos recogido más de 1.450 toneladas del fondo del océano desde 2015 gracias al apoyo de más de 4.500 pescadores. Ayúdanos a seguir limpiando nuestros océanos.',
 'El abrigo Velan integra la tecnología innovadora dope dye que permite un proceso de teñido sin agua. En

In [18]:
nombres_chaqueta = [chaqueta.lower().split(" ")[:2] for chaqueta in nombres_done]

nombres_chaqueta = [f"{i[0]} {i[1].capitalize()}" for i in nombres_chaqueta]
# listas_unidas = [" ".join(sublista) for sublista in nombres_chaqueta]

print(nombres_chaqueta)

['chaqueta Venue', 'chaqueta Venue', 'chaqueta Grays', 'chaqueta Grays', 'abrigo Livorno', 'abrigo Livorno', 'abrigo Velan', 'abrigo Velan', 'abrigo Pasu', 'abrigo Pasu', 'chaqueta Venue', 'chaqueta Venue', 'chaqueta Vinson', 'chaqueta Vinson', 'abrigo Iceberg', 'abrigo Iceberg', 'abrigo Parko', 'abrigo Parko', 'chaqueta Vinson', 'chaqueta Vinson', 'chaqueta Cartes', 'chaqueta Cartes', 'abrigo New', 'abrigo New', 'chaqueta Grays', 'chaqueta Grays', 'abrigo Iceberg', 'abrigo Iceberg', 'abrigo Parko', 'abrigo Parko', 'chaqueta Asp', 'chaqueta Asp', 'chaqueta Asp', 'chaqueta Asp', 'chaqueta Cronin', 'chaqueta Cronin', 'abrigo Parko', 'abrigo Parko', 'chaqueta Lawu', 'chaqueta Lawu', 'chaleco Aoraki', 'chaleco Aoraki', 'chaqueta Alpen', 'chaqueta Alpen', 'chaqueta Beret', 'chaqueta Beret', 'chaqueta Beret', 'chaqueta Beret', 'chaleco Aoraki', 'chaleco Aoraki', 'chaqueta Lawu', 'chaqueta Lawu', 'chaqueta Beret', 'chaqueta Beret', 'chaqueta Beret', 'chaqueta Beret', 'chaleco Sadel', 'chaleco

In [19]:
nombre_descripcion_dict = {}
for nombre in nombres_chaqueta:
    for des in descripcion:
        if nombre in des:
            nombre_descripcion_dict[nombre] = des
            pass  # Detener la búsqueda si ya se ha encontrado una coincidencia

nombre_descripcion_dict = {key.upper(): value for key, value in nombre_descripcion_dict.items()}


print(nombre_descripcion_dict)

{'CHAQUETA VENUE': 'La chaqueta Venue está hecha de poliéster reciclado que procede de botellas de plástico a las que damos una segunda vida. Lleva capucha incorporada ajustable y entre sus propiedades técnicas destaca que es impermeable y transpirable; el abrigo perfecto para los días de lluvia.', 'CHAQUETA GRAYS': 'La chaqueta Grays es monomaterial y está hecha de 100% poliéster reciclado procedente de botellas de plástico. Además, es transpirable y repelente al agua. Una chaqueta muy cómoda y perfecta para este invierno con la que combinarás estilo y sostenibilidad.', 'ABRIGO LIVORNO': 'El abrigo Livorno está hecho con nuestro Hilo del Mar procedente de botellas de plástico recicladas del fondo del mar a través del proyecto Upcycling the Oceans de la Fundación Ecoalf. Hemos recogido más de 1.450 toneladas del fondo del océano desde 2015 gracias al apoyo de más de 4.500 pescadores. Ayúdanos a seguir limpiando nuestros océanos.', 'ABRIGO VELAN': 'El abrigo Velan integra la tecnología 

In [36]:
df_ecoalf["nombre"] = df_ecoalf["nombre"].drop_duplicates()
df_ecoalf = df_ecoalf.dropna()

len(df_ecoalf)

36

In [35]:
# Función para buscar las claves del diccionario en el nombre y aplicar la descripción
def aplicar_descripcion(nombre, dic):
    # Recorre todas las claves del diccionario
    for key, descripcion in dic.items():
        if key in nombre:  # Si la clave está contenida en el nombre
            return descripcion  # Retorna la descripción
    return nombre  # Si no se encuentra, regresa el nombre original

# Aplicar la función a la columna 'nombre' del DataFrame usando lambda para pasar el diccionario
df_ecoalf['descripcion'] = df_ecoalf['nombre'].apply(lambda x: aplicar_descripcion(x, nombre_descripcion_dict))

# Mostrar el DataFrame con la nueva columna
len(df_ecoalf)


36

In [22]:
len(df_ecoalf)

36

In [30]:
tejido = [i for i in tejido if "Tejido" in i]

tejido

['Tejido principal: 100% poliéster reciclado',
 'Tejido principal: 100% poliéster reciclado',
 'Tejido principal: 58% Hilo del Mar (poliéster reciclado) / 42% poliéster reciclado',
 'Tejido principal: 100% poliéster reciclado',
 'Tejido principal: 100% poliéster reciclado',
 'Tejido principal: 100% poliéster reciclado',
 'Tejido principal: 58% Hilo del Mar (poliéster reciclado) / 42% poliéster reciclado',
 'Tejido principal: 58% Hilo del Mar (poliéster reciclado) / 42% poliéster reciclado',
 'Tejido principal: 100% poliéster reciclado',
 'Tejido principal: 100% poliéster reciclado',
 'Tejido principal: 90% poliéster reciclado/ 10% poliéster',
 'Tejido principal: 100% poliéster reciclado',
 'Tejido principal: 58% Hilo del Mar (poliéster reciclado) / 42% poliéster reciclado',
 'Tejido principal: 58% Hilo del Mar (poliéster reciclado) / 42% poliéster reciclado',
 'Tejido principal: 100% poliéster reciclado',
 'Tejido principal: 100% poliéster reciclado',
 'Tejido principal: 100% poliéster

In [37]:
# Paso 1: Contar la frecuencia de cada material
frecuencias = Counter(tejido)

# Paso 2: Calcular la cantidad deseada para cada material en una lista de 74 elementos
target_total = 36
proporciones = {material: int(count * target_total / len(tejido)) for material, count in frecuencias.items()}

# Paso 3: Ajustar el total a 74 exactamente, en caso de desviaciones
ajuste = target_total - sum(proporciones.values())
if ajuste > 0:
    # Si faltan elementos, incrementar algunos en +1 hasta llegar a 74
    for material in random.sample(list(proporciones.keys()), ajuste):
        proporciones[material] += 1
elif ajuste < 0:
    # Si hay elementos de más, reducir algunos en -1 hasta llegar a 74
    for material in random.sample(list(proporciones.keys()), -ajuste):
        proporciones[material] -= 1

# Paso 4: Construir la lista final de materiales
materiales_reducidos = []
for material, count in proporciones.items():
    # Usamos sample si count <= frecuencia real, de lo contrario usamos choices para evitar error
    muestras_material = (random.sample([m for m in tejido if m == material], count)
                         if count <= frecuencias[material]
                         else random.choices([m for m in tejido if m == material], k=count))
    materiales_reducidos.extend(muestras_material)

print(len(materiales_reducidos))  # Debe imprimir exactamente 74
print(materiales_reducidos)

36
['Tejido principal: 100% poliéster reciclado', 'Tejido principal: 100% poliéster reciclado', 'Tejido principal: 100% poliéster reciclado', 'Tejido principal: 100% poliéster reciclado', 'Tejido principal: 100% poliéster reciclado', 'Tejido principal: 100% poliéster reciclado', 'Tejido principal: 100% poliéster reciclado', 'Tejido principal: 100% poliéster reciclado', 'Tejido principal: 100% poliéster reciclado', 'Tejido principal: 100% poliéster reciclado', 'Tejido principal: 100% poliéster reciclado', 'Tejido principal: 100% poliéster reciclado', 'Tejido principal: 100% poliéster reciclado', 'Tejido principal: 100% poliéster reciclado', 'Tejido principal: 100% poliéster reciclado', 'Tejido principal: 100% poliéster reciclado', 'Tejido principal: 100% poliéster reciclado', 'Tejido principal: 100% poliéster reciclado', 'Tejido principal: 100% poliéster reciclado', 'Tejido principal: 100% poliéster reciclado', 'Tejido principal: 100% poliéster reciclado', 'Tejido principal: 100% poliés

In [38]:

df_ecoalf["genero"] = genero

df_ecoalf["mateiral"] = materiales_reducidos

In [41]:
!pip install fsspec

In [42]:
df_ecoalf.to_csv("C://Users//Víctor//Desktop//HACKIO//Proyectos//ProyectoMinimalism//datos//df_ecoalf.csv")